# Import libraries

In [1]:
import pandas as pd
import googlemaps
import os
import pymongo
import requests
import json
import geopandas as gpd
import shapely.geometry
from pymongo import MongoClient, GEOSPHERE
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
import sys
sys.path.append('../')
import src.cleaning as cl

In [2]:
client = MongoClient('localhost:27017')
db = client.get_database('ironhack')

# Create mongo collections per city

In [3]:
load_dotenv()

True

In [11]:
g_API = os.getenv('key')
gmaps_key = googlemaps.Client(key=g_API)

In [ ]:
"""
List of attributes to search per city:
-bar // r = 1,000mts
-airport // r = 30,000mts
-school // r = 10,000mts
""";

## Madrid

In [83]:
mad_lat = cl.geocode('Madrid')['coordinates'][0]
mad_lng = cl.geocode('Madrid')['coordinates'][1]

In [84]:
mad_bar = cl.nearby(mad_lat, mad_lng, 1000, 'bar')
mad_school = cl.nearby(mad_lat, mad_lng, 10000, 'school')
mad_airport = cl.nearby(mad_lat, mad_lng, 30000, 'airport')
df_mad = pd.concat([mad_bar, mad_school, mad_airport]).reset_index(drop=True)

In [ ]:
# Exportar a mongo

In [85]:
df_mad = gpd.GeoDataFrame(df_mad, geometry=gpd.points_from_xy(df_mad.longitude, df_mad.lattitude))
df_mad.sample(3)

,types,name,longitude,lattitude,geometry
57,airport,Aegean Airlines,-3.569499,40.467634,POINT (-3.56950 40.46763)
19,bar,Bar Rama,-3.636650,40.396986,POINT (-3.63665 40.39699)
27,school,Hastings School - Azulinas (Primary),-3.683805,40.464174,POINT (-3.68380 40.46417)


In [86]:
df_mad['geometry']=df_mad['geometry'].apply(lambda x:shapely.geometry.mapping(x))

/opt/homebrew/Caskroom/miniconda/base/envs/ironhack/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [87]:
collectionMadrid = client.ironhack['Madrid'] #create collection name
collectionMadrid.create_index([("geometry", GEOSPHERE)]) #create collection index
df_mad = df_mad.to_dict(orient='records') # change pd.DataFrame format to dictionary
collectionMadrid.insert_many(df_mad) #insert data into collection

## Vancouver

In [77]:
van_lat = cl.geocode('Vancouver')['coordinates'][0]
van_lng = cl.geocode('Vancouver')['coordinates'][1]

In [78]:
van_bar = cl.nearby(van_lat, van_lng, 1000, 'bar')
van_school = cl.nearby(van_lat, van_lng, 10000, 'school')
van_airport = cl.nearby(van_lat, van_lng, 30000, 'airport')
df_van = pd.concat([van_bar, van_school, van_airport]).reset_index(drop=True)

In [79]:
df_van = gpd.GeoDataFrame(df_van, geometry=gpd.points_from_xy(df_van.longitude, df_van.lattitude))
df_van.sample(3) #insert geo column

,types,name,longitude,lattitude,geometry
27,school,York House School,-123.141172,49.248905,POINT (-123.14117 49.24891)
41,airport,YVR-Airport,-123.178273,49.194209,POINT (-123.17827 49.19421)
44,airport,Vancouver VORTAC YVR 115.9,-123.149102,49.077324,POINT (-123.14910 49.07732)


In [80]:
df_van['geometry']=df_van['geometry'].apply(lambda x:shapely.geometry.mapping(x))

/opt/homebrew/Caskroom/miniconda/base/envs/ironhack/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [81]:
collectionVancouver = client.ironhack['Vancouver'] #create collection name
collectionVancouver.create_index([("geometry", GEOSPHERE)]) #create collection index
df_van = df_van.to_dict(orient='records') # change pd.DataFrame format to dictionary
collectionVancouver.insert_many(df_van) #insert data into collection

## Trentino-Alto Adige

In [13]:
tren_lat = cl.geocode('Trentino-Alto Adige')['coordinates'][0]
tren_lng = cl.geocode('Trentino-Alto Adige')['coordinates'][1]

In [72]:
tren_bar = cl.nearby(tren_lat, tren_lng, 1000, 'bar')
tren_school = cl.nearby(tren_lat, tren_lng, 10000, 'school')
tren_airport = cl.nearby(tren_lat, tren_lng, 30000, 'airport')
df_tren = pd.concat([tren_bar, tren_school, tren_airport]).reset_index(drop=True)

In [73]:
df_tren = gpd.GeoDataFrame(df_tren, geometry=gpd.points_from_xy(df_tren.longitude, df_tren.lattitude))
df_tren.sample(3)

,types,name,longitude,lattitude,geometry
38,secondary_school,Landesberufsschule Handel und Grafik Johannes ...,11.329987,46.482701,POINT (11.32999 46.48270)
32,secondary_school,Mittelschule Deutschnofen,11.427888,46.414612,POINT (11.42789 46.41461)
20,secondary_school,Waldorf Ober- & Mittelstufe Bozen,11.325133,46.473109,POINT (11.32513 46.47311)


In [74]:
df_tren['geometry']=df_tren['geometry'].apply(lambda x:shapely.geometry.mapping(x))

/opt/homebrew/Caskroom/miniconda/base/envs/ironhack/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [75]:
collectionTrentino = client.ironhack['Trentino-Alto Adige'] #create collection name
collectionTrentino.create_index([("geometry", GEOSPHERE)]) #create collection index
df_tren = df_tren.to_dict(orient='records') # change pd.DataFrame format to dictionary
collectionTrentino.insert_many(df_tren) #insert data into collection